# Encoder–Decoder 구조

- Encoder–Decoder 구조는 어떤 형태의 입력 시퀀스를 받아 **의미를 해석**한 뒤, 새로운 **출력 시퀀스를 생성**해야 하는 거의 모든 AI 문제를 해결하는 딥러닝 모델 구조다.
- 이 구조는 Encoder와 Decoder 두개의 딥러닝 모델을 연결한 구조로 **입력 데이터를 하나의 표현으로 압축한 뒤, 이를 다시 출력 데이터로 변환하는 방식**으로 동작한다.

- **Encoder Network**
  - 입력 데이터를 해석(이해)하는 역할을 수행한다.
  - 입력 시퀀스에 담긴 의미적 정보를 하나의 고정된 벡터 형태로 요약한다.

- **Decoder Network**
  - Encoder가 생성한 요약 정보를 바탕으로 최종 출력을 생성한다.
  - 즉, Encoder의 “이해 결과”를 이용해 새로운 시퀀스를 만들어낸다.

## Seq2Seq (Sequence-to-Sequence)

Seq2Seq 모델은 **Encoder–Decoder 구조를 RNN(Recurrent Neural Network) 계열에 적용한 대표적인 시퀀스 변환 모델**이다.  
입력과 출력이 모두 “시퀀스(sequence)” 형태라는 점에서 *Sequence-to-Sequence*라는 이름이 붙었다.

### Encoder의 역할: 입력 시퀀스 이해 및 Context Vector 생성

Encoder는 입력으로 들어온 **전체 시퀀스**(sequence)를 순차적으로 처리한 뒤,  그 의미를 **하나의 고정 길이 벡터**(Vector)로 압축하여 출력한다.  
이 벡터를 **Context Vector**(컨텍스트 벡터)라고 한다.
- **Context Vector란?**  
  - 입력 시퀀스 전체의 의미, 문맥, 핵심 정보를 요약해 담고 있는 벡터 표현이다.
  - **기계 번역**(Machine Translation)의 경우  
    - 번역할 원문 문장에서 **번역 결과를 생성하는 데 필요한 핵심 의미 정보**(feature)
  - **챗봇**(Chatbot)의 경우  
    - 사용자가 입력한 질문에서 **적절한 답변을 생성하는 데 필요한 의미 정보**(feature)

### Decoder의 역할: Context Vector를 바탕으로 출력 시퀀스 생성

Decoder는 Encoder가 출력한 **Context Vector를 입력으로 받아**, 이를 바탕으로 **목표 출력 시퀀스**를 한 토큰(token)씩 순차적으로 생성한다.

- **기계 번역**(Machine Translation)의 경우  
  - 입력 문장의 의미를 반영한 **번역 문장**을 생성한다.
- **챗봇**(Chatbot)  
  - 질문에 대한 **자연스러운 답변 문장**을 생성한다.

Decoder는 매 시점(time step)마다
  - 이전에 생성한 단어
  - 그리고 Context Vector에 담긴 입력 문맥
을 함께 고려하여 다음 단어를 예측한다.


![seq2seq](figures/seq2seq.png)

# Seq2Seq 를 이용한 Chatbot 모델 구현
- Encoder를 이용해 질문의 특성을 추출하고 Decoder를 이용해 답변을 생성한다.

# Chatbot Dataset

- https://github.com/songys/Chatbot_data
- columns
    - Q: 질문
    - A: 답
    - label: 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2
- **Download**

![dataset](figures/chatbot.png)

# Chatbot Dataset Loading 및 확인

## 데이터셋 다운로드 및 확인

In [1]:
import requests
import os

url = "https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv"
res = requests.get(url)
if res.status_code == 200:
    with open("data/chatbot_data.csv", "wt", encoding="utf-8") as fw:
        fw.write(res.text)
else:
    print("다운실패:", res.status_code)

In [2]:
import pandas as pd

df = pd.read_csv("data/chatbot_data.csv", encoding="utf-8")
df.shape

(11823, 3)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [4]:
df.drop(columns='label', inplace=True)

In [5]:
df.head(10)

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.


In [6]:
df.tail(10)

,Q,A
11813,회사에 좋아하는 남자가 생겼어 어떡하지?,사랑하기 힘든 관계인가봐요.
11814,회사에서 어떤 사람이랑 자꾸 눈 마추쳐.,눈 마주치는 게 우연인지 잘 살펴 보세요.
11815,회식 중이라고 하는데 연락이 안돼.,정신 없이 바쁠지도 몰라요. 조금만 더 기다려보고 물어보는게 좋을 것 같아요.
11816,회식하는데 나만 챙겨줘. 썸임?,호감이 있을 수도 있어요. 그렇지만 조금 더 상황을 지켜보세요.
11817,후회 없이 사랑하고 싶어,진심으로 다가가 보세요.
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.


In [7]:
df.isnull().sum()

Q    0
A    0
dtype: int64

# Dataset, DataLoader 정의

## Tokenization

### Subword방식

In [8]:
# 토큰화를 위해서 문장을 q + a 형식으로 만든다.
# 어휘사전을 만들때 Q와 A에 있는 모든 단어들이 다 들어가게 하기 위해.
question_texts = df['Q'] # Series (str)
answer_texts = df['A'] # Series (str)

all_texts = list(question_texts+" "+answer_texts)  # series + 문자열 + series  (원소단위 연산)

In [9]:
all_texts[:10]

['12시 땡! 하루가 또 가네요.',
 '1지망 학교 떨어졌어 위로해 드립니다.',
 '3박4일 놀러가고 싶다 여행은 언제나 좋죠.',
 '3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠.',
 'PPL 심하네 눈살이 찌푸려지죠.',
 'SD카드 망가졌어 다시 새로 사는 게 마음 편해요.',
 'SD카드 안돼 다시 새로 사는 게 마음 편해요.',
 'SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요.',
 'SNS 시간낭비인 거 아는데 매일 하는 중 시간을 정하고 해보세요.',
 'SNS 시간낭비인데 자꾸 보게됨 시간을 정하고 해보세요.']

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer(
    BPE(unk_token="<unk>")
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size=10_000, # 최대 어휘수
    min_frequency=5,   # 어휘사전에 등록할 단어의 최소 빈도수. (5회 이상은 나와야 등록.)
    continuing_subword_prefix='##', # 연결 subword 앞에 붙일 접두어. cowork : co + ##work
    special_tokens=["<pad>","<unk>","<sos>"] # <sos> 문장의 시작을 의미하는 특수토큰.
)

tokenizer.train_from_iterator(all_texts, trainer=trainer)

In [11]:
print("총 어휘수:", tokenizer.get_vocab_size())

총 어휘수: 7040


In [12]:
encode = tokenizer.encode("오늘 날씨가 너무 좋습니다. 이런 날씨에 뭘 하면 좋을까요?")

In [13]:
encode.tokens

['오늘', '날씨가', '너무', '좋습니다', '.', '이런', '날씨', '##에', '뭘', '하면', '좋을까요', '?']

In [15]:
encode.ids

[2290, 3852, 2258, 5913, 8, 2752, 2841, 1267, 527, 2530, 5532, 20]

In [20]:
tokenizer.id_to_token(1500)  # id로 토큰문자열 조회
tokenizer.token_to_id("하면")# 토큰 문자열로 id(정수)를 조회

2530

### Tokenizer 저장

In [21]:
tokenizer.save("saved_models/chatbot_bpe.json")

In [22]:
load_tokenizer = Tokenizer.from_file("saved_models/chatbot_bpe.json")

## Dataset, DataLoader 정의


### Dataset 정의 및 생성
- 모든 문장의 토큰 수는 동일하게 맞춰준다.
    - DataLoader는 batch 를 구성할 때 batch에 포함되는 데이터들의 shape이 같아야 한다. 그래야 하나의 batch로 묶을 수 있다.
    - 문장의 최대 길이를 정해주고 **최대 길이보다 짧은 문장은 `<PAD>` 토큰을 추가**하고 **최대길이보다 긴 문장은 최대 길이에 맞춰 짤라준다.**

In [23]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "mps"

In [30]:
class ChatbotDataset(Dataset):

    def __init__(self, question_texts, answer_texts, max_length, tokenizer):
        """
        Args:
            question_texts (list[str]): 질문 text 리스트. ["질문1", "질문2", ..]
            answer_texts (list[str]): 답변 text 리스트. ["답변1", "답변2", ...]
            max_length (int): 개별 문장의 최대 토큰 수
            tokenizer (Tokenzier)
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        # "질문" -> Tensor(토큰 id)
        self.question_texts = [self.__process_sequence(q) for q in question_texts]
        self.answer_texts = [self.__process_sequence(a) for a in answer_texts]

    def __pad_token_sequence(self, token_sequence):
        """
        token_sequence를 self.max_length 길이에 맞추는 메소드.
        max_length보다 적으면 <pad>를 추가, 크면 잘라낸다.
        Args:
            token_sequence (list[int]): 한 문장의 토큰 id 리스트. [2334, 7100, 257, ..]
        Returns:
            list[int]: 길이를 max_length에 맞춘 토큰 id 리스트
        """
        pad_token = self.tokenizer.token_to_id('<pad>')
        seq_length = len(token_sequence)
        if seq_length > self.max_length: #잘라내기
            result = token_sequence[:self.max_length]
        else: # <pad> 추가 (padding 처리)
            result = token_sequence + [pad_token] * (self.max_length - seq_length)
    
        return result
    
    def __process_sequence(self, text):
        """
        한 문장(text-str)을 받아서 token화 한 뒤 max_length에 개수를 맞춰서 반환.
        max_length에 맞추는 작업은 __pad_token_sequence() 를 이용
        Args:
            text (str): 토큰화할 문장
        Returns:
            torch.Tensor[int64]: 토큰화한 토큰 id 리스트
        """
        encode = self.tokenizer.encode(text)
        token_ids = encode.ids # "나는 학생이다." -> [4020, 1003, 3932]
        # [4020, 1003, 3932] -> [4020, 1003, 3932, 0, 0, 0 ] 패딩 처리.
        return torch.tensor(self.__pad_token_sequence(token_ids), dtype=torch.int64)


    def __len__(self):
        return len(self.question_texts)

    def __getitem__(self, index):
        """
        index의 (question, answer) 쌍을 반환.
        Args:
            index (int) : 몇번 질문-답변 쌍인지 index
        Return:
            tuple[Tensor(int64), Tensor(int64)]
        """
        q = self.question_texts[index]
        a = self.answer_texts[index]
        return q, a

In [47]:
# max length. 가장 긴 문장의 토큰 수
max([len(tokenizer.encode(sent).ids) for sent in question_texts])

21

In [48]:
max([len(tokenizer.encode(sent).ids) for sent in answer_texts])

29

In [49]:
max_length = 29
dataset = ChatbotDataset(
    list(question_texts),
    list(answer_texts),
    max_length,
    tokenizer
)

In [50]:
len(dataset)

11823

In [51]:
dataset[0]

(tensor([  10, 1747, 1400,  368,    3,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]),
 tensor([6119,  378,   47, 2252,    8,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]))

In [54]:
len(dataset)

11823

### Trainset / Testset 나누기
train : test = 0.95 : 0.05

In [58]:
train_size = int(len(dataset) * 0.95)
test_size = len(dataset) - train_size
train_size, test_size

(11231, 592)

In [59]:
train_set, test_set = random_split(dataset, [train_size, test_size])

In [61]:
type(dataset), type(train_set)

(__main__.ChatbotDataset, torch.utils.data.dataset.Subset)

In [62]:
len(train_set), len(test_set)

(11231, 592)

### DataLoader 생성

In [63]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=64)

In [64]:
len(train_loader), len(test_loader)

(175, 10)

# 모델 정의

## Seq2Seq 모델 정의
- Seq2Seq 모델은 Encoder와 Decoder의 입력 Sequence의 길이와 순서가 자유롭기 때문에 챗봇이나 번역에 이상적인 구조다.
    - 단일 RNN은 각 timestep 마다 입력과 출력이 있기 때문에 입/출력 sequence의 개수가 같아야 한다.
    - 챗봇의 질문/답변이나 번역의 대상/결과 문장의 경우는 사용하는 어절 수가 다른 경우가 많기 때문에 단일 RNN 모델은 좋은 성능을 내기 어렵다.
    - Seq2Seq는 **입력처리(질문,번역대상)처리 RNN과 출력 처리(답변, 번역결과) RNN 이 각각 만들고 그 둘을 연결한 형태로 길이가 다르더라도 상관없다.**

## Encoder
Encoder는 하나의 Vector를 생성하며 그 Vector는 **입력 문장의 의미**를 N 차원 공간 저장하고 있다. 이 Vector를 **Context Vector** 라고 한다.    
![encoder](figures/seq2seq_encoder.png)

In [69]:
# a = nn.Embedding(100, 5, padding_idx=2)
# a.weight

In [79]:
class Encoder(nn.Module):

    def __init__(
            self, 
            vocab_size: int, # 총 어휘수
            embedding_dim: int, # Embedding Vector의 차원
            hidden_size: int,   # GRU의 HIDDEN 개수
            bidirectional: bool = True, # GRU의 양방향 여부
            num_layers: int = 1, # GRU의 layer stack 수
            dropout: float = 0.2   # dropout 비율
    ):
        super().__init__()
        self.vocab_size = vocab_size
        # X -> (Embedding Model) -> (GRU) -> Context Vector -> (Decoder)
        # Encoder의 목적은 (질문의) Context Vector 를 추출하는 것이 목적.
        self.embedding = nn.Embedding(
            vocab_size,
            embedding_dim, # (vocab_size x embedding_dim)
            padding_idx=0
        )

        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout if num_layers > 1 else 0.0
        )

    def forward(self, X):
        # X.shpae [batch, seq_length]
        embedding_vector = self.embedding(X) # [batch, seq_length, emb_dim]
        embedding_vector = embedding_vector.transpose(1, 0) # [seq_length, batch, emb_dim]
        out, hidden = self.gru(embedding_vector) 
        # out: 모든 timestep의 hidden state, hidden: 마지막 timestep의 hidden state
        return out, hidden


In [80]:
from torchinfo import summary

dummy_input = torch.randint(10, size=(64, 20), dtype=torch.int64)
summary(Encoder(1000, 100, 20), input_data=dummy_input)

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [20, 64, 40]              --
├─Embedding: 1-1                         [64, 20, 100]             100,000
├─GRU: 1-2                               [20, 64, 40]              14,640
Total params: 114,640
Trainable params: 114,640
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 25.14
Input size (MB): 0.01
Forward/backward pass size (MB): 1.43
Params size (MB): 0.46
Estimated Total Size (MB): 1.90

## Decoder
- Encoder의 출력(context vector)를 받아서 번역 결과 sequence를 출력한다.
- Decoder는 매 time step의 입력으로 **이전 time step에서 예상한 단어와 hidden state값이** 입력된다.
- Decoder의 처리결과 hidden state를 Estimator(Linear+Softmax)로 입력하여 **입력 단어에 대한 번역 단어가 출력된다.** (이 출력단어가 다음 step의 입력이 된다.)
    - Decoder의 첫 time step 입력은 문장의 시작을 의미하는 <SOS>(start of string) 토큰이고 hidden state는 context vector(encoder 마지막 hidden state) 이다.

![decoder](figures/seq2seq_decoder.png)

In [ ]:
class Decoder(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hiddem_size,
                 bidirectional=False,
                 dropout=0.2):
        super.__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.gru = nn.GRU(
            embedding_dim,
            hidden_size,
            num_layers=num_layers
            dropout=dropout if num_layers > 1 else 0.0
        )
        
        self.classifier = nn.Linear(
            hidden_size,
            vocab_size
        )
    def forword(self, X, hidden):
        """
        X: 한 개 토큰 / shape: [batch]
        hidden: 이전 처리 hidden state / 첫번째 timestep: Encorder의 context vector
                [seq_length: 1, batch, hidden_size]
        """
        embedding_vector = self.embedding(X) # 입력: int64, [batch, seq_length]
        # [batch, seq_length, embedding_dim] -> [seq_length(1), batch, embedding_dim]
        embedding_vector = embedding_vector.transpose(1, 0)

        # [seq_length(1), batch, embedding_dim] -> out, hidden
        # out(모든 timestep의 hidden state 모음) : [seq_length(1), batch_size, hidden_size]
        # hidden(마지막 timestep의 hidden state): [num_laers]
        out, hidden = self.gru(embedding_vector)

        # Linear(분류기) 넣어서 다음 단어를 예측
        last_out = self.classifier(out[-1])

        # last_out: 다음 단어일 확률, [batch, vocab_size]
        # hidden: 다음 단어를 예측할 때 넣어줄 context vector(hidden state)
        return last_out, hidden

In [ ]:
dummy_input = torch.ones([64], dtype=torch.int64) # 64: batch
dummy_hidden = torch.ones((1, 64, 200), dtype=torch.float32)
# hidden shape : [1 - gru layer 개수, 64: batch, 200: hidden_size]
# gru layer 개수 : num _layers * 2 if bidirectional else 1

dummy_decoder = Decoder(1000, 200, 256, num_layers=1)

In [ ]:
next_word, hidden = dummy_decoder(dummy_input, dummy_hidden)

In [ ]:
print(next_word.shape)
print(hidden.shape)

In [ ]:
next_word.max(dim=-1).indices

## Seq2Seq 모델

- Encoder - Decoder 를 Layer로 가지며 Encoder로 질문의 feature를 추출하고 Decoder로 답변을 생성한다.

### Teacher Forcing
- **Teacher forcing** 기법은, RNN계열 모델이 다음 단어를 예측할 때, 이전 timestep에서 예측된 단어를 입력으로 사용하는 대신 **실제 정답 단어(ground truth) 단어를** 입력으로 사용하는 방법이다.
    - 모델은 이전 시점의 출력 단어를 다음 시점의 입력으로 사용한다. 그러나 모델이 학습할 때 초반에는 정답과 많이 다른 단어가 생성되어 엉뚱한 입력이 들어가 학습이 빠르게 되지 않는 문제가 있다.
- **장점**
    - **수렴 속도 증가**: 정답 단어를 사용하기 때문에 모델이 더 빨리 학습할 수있다.
    - **안정적인 학습**: 초기 학습 단계에서 모델의 예측이 불안정할 때, 잘못된 예측으로 인한 오류가 다음 단계로 전파되는 것을 막아줍니다.
- **단점**
    - **노출 편향(Exposure Bias) 문제:** 실제 예측 시에는 정답을 제공할 수 없으므로 모델은 전단계의 출력값을 기반으로 예측해 나가야 한다. 학습 과정과 추론과정의 이러한 차이 때문에 모델의 성능이 떨어질 수있다.
        - 이런 문제를 해결하기 학습 할 때 **Teacher forcing을 random하게 적용하여 학습시킨다.**
![seq2seq](figures/seq2seq.png)

In [ ]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder.to(device)
        self.decoder = decoder.to(device)
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_rate=0.99):
        """
        Args:
            inputs: 질문. (batch, seq_length)
            outputs: 답변(정답). (batch, seq_length) -> teacher forcing 때 사용
            teacher_forcing_rate: teacher forcing은 random하게 적용, 적용될 확률
        """
        # 질문과 답변이 1차원일 경우 2차원으로 reshape
        # [batch] -> [batch(1), seq_length]
        if inputs.dim() == 1:
            inputs = inputs.unsqueeze(0)
        if outputs.dim() == 1:
            outputs = outputs.unsqueeze(0)

        batch_size, output_length = outputs.shape
        output_vocab_size = self.encoder.vocab_size

        ##############################################
        # 생성된 문장을 저장할 tensor를 생성
        # (seq_length, batch_size, vocab_size)
        # [나는, 학생, 이다]
        # [
        #  vocab_size의 각 단어가 "나는"일 확률
        #  vocab_size의 각 단어가 "학생"일 확률
        #  vocab_size의 각 단어가 "이다"일 확률
        # ]
        ##############################################
        predicted_outputs = torch.zeros(output_length, batch_size, output_vocab_size).to(self.device)


        ###################################################################
        # 추론
        # 1. encoder를 이용해서 context vector 추출(한번에 처리)
        # 2. decoder를 이용해서 답변 문장을 생성(개별 토큰별로 생성)
        ###################################################################
        # encoder를 이용해서 context vector 추출
        encoder_out, _ = self.encoder(inputs) # encoder_out: 전체 hidden state 모음 

        # context vector == encoder_out[-1] ==> Decoder에 첫번째 timestep의 hidden으로 입력
        decoder_hidden = encoder_out[-1].unsqueeze(0)
        # decoder에 입력할 첫번째 timestep의 값: <sos> 토큰 id
        decoder_input = torch.full([batch_size], fill_value=SOS_TOKEN, device=self.device)

        ########################################
        # Decoder를 이용해서 한 단어(토큰)씩 생성
        ########################################
        for t in range(output_length):
            decoder_out, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            predicted_outputs[t] = decoder_out # t번째 예측 단어

            # 다음 timestep의 input을 생성 (decoder_input값을 생성)
            # teacher forcing 적용      -> t번째 정답 토큰
            #                 적용 안함 -> Decoder가 생성한 decoder_out의 token id 값

            teacher_forcing = techer_forcing_rate > random.random() 
            techer_forcing_rate *= 0.99 # 반복하면서 teacher focing 적용 확률을 죽여 나간다

            top1 = decoder_out.argmax(dim=-1) # 다음 단어일확률아 가장 높은 단어의 토큰 id
            decoder_out

# 학습

## 모델생성

## loss함수, optimizer

## train/evaluation 함수 정의

### train 함수정의

In [ ]:
def train(model, dataloader, optimizer, loss_fn, device, teacher_forcing_rate=0.9):
    medel.train()
    train_loss = 0.0

    for X,y in dataloader:
        X,y = X.to(device), y.to(device)
        pred = model(X, y, teacher_forcing_rate) # seq2seq
        # pred: [batch, seq_length, vocab_size]
        # pred: [batch * seq_length, vocab_size]
        y_hat = pred.reshape(-1, pred.shape[2]) # loss 계산을 위해서
        y = y. reshape(-1) # [batch, seq_length] -> [batch*seq_length]
        # CrossEntropyLoss입력: 정답 - [batch,], 추론: [batch, class개수-vocab_size]
        loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()

    return train_loss / len(dataload)

In [ ]:
@torch.no_grad()
def eval(model, dataloader, loss_fn , device):
    model.eval()
    eval_loss = 0.0

    for X,y in dataloader:
        X, y = X.to(device), y.to(device)
        

### Test 함수

### Training

# 결과확인

- Sampler:
    -  DataLoader가 Datatset의 값들을 읽어서 batch를 만들때 index 순서를 정해주는 객체.
    -  DataLoader의 기본 sampler는 SequentialSampler 이다. shuffle=True 일경우 RandomSampler: 랜덤한 순서로 제공.

# 학습모델을 이용한 대화